In [8]:
Sarcasm_Path = '/sarcasm_headlines.csv'

In [9]:
import pandas as pd

def read_and_view_csv(Sarcasm_Path):
    sarcasm_headlines = pd.read_csv(Sarcasm_Path)
    sarcasm_headlines.dropna(inplace=True)
    print("Shape= ", sarcasm_headlines.shape)

    print("Columns names:", sarcasm_headlines.columns.tolist())
    return sarcasm_headlines
sarcasm_headlines= read_and_view_csv(Sarcasm_Path)

Shape=  (26600, 3)
Columns names: ['headline', 'is_sarcastic', 'Cleaned_Headlines']


In [10]:
sarcasm_headlines = sarcasm_headlines[['is_sarcastic','Cleaned_Headlines']] #take cleaned headlines content and the output variables.

In [11]:
sarcasm_headlines

,is_sarcastic,Cleaned_Headlines
0,0,former versace store clerk sues secret black c...
1,0,roseanne revival catches thorny political mood...
2,1,mom starting fear son web series closest thing...
3,1,boehner wants wife listen come alternative deb...
4,0,j k rowling wishes snape happy birthday magica...
...,...,...
26597,0,american politics moral free fall
26598,0,america best hikes
26599,0,reparations obama
26600,0,israeli ban targeting boycott supporters raise...


In [12]:
import warnings as WARN
WARN.filterwarnings('ignore')

In [13]:
sarcasm_headlines['is_sarcastic'].value_counts()

,count
is_sarcastic,
0,14949
1,11651


In [14]:
X = sarcasm_headlines['Cleaned_Headlines']
y = sarcasm_headlines['is_sarcastic']

In [15]:
!pip install transformers

In [2]:
import torch

from transformers import RobertaTokenizer
from transformers import RobertaModel

In [3]:
Tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
Roberta_Model = RobertaModel.from_pretrained('roberta-base')

Roberta_Model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [16]:
# Tokenize and encode the text content

def Encoder(headlines):
    headlines = headlines.tolist()
    Encoder = Tokenizer(headlines, truncation=True, padding=True, max_length=10, return_tensors='pt')
    with torch.no_grad():
        Embedder = Roberta_Model(**Encoder).last_hidden_state
    return Embedder.mean(dim=1).numpy()

In [17]:
X = Encoder(X)
print(X)

[[ 0.12038483  0.03733467 -0.03433609 ... -0.03632325 -0.02576233
   0.05796341]
 [-0.02295212  0.07673141  0.08296929 ... -0.04027725 -0.05888848
   0.01016079]
 [ 0.00854696  0.0184384  -0.00297035 ... -0.31569093 -0.11779124
  -0.01756739]
 ...
 [-0.01025138 -0.01395688  0.05522687 ... -0.07565024  0.03816073
   0.09149651]
 [ 0.04028695  0.0274882  -0.01897702 ...  0.01969844 -0.02070792
   0.04509629]
 [-0.03837262 -0.05316873 -0.02269211 ...  0.02579583 -0.04568146
   0.13098168]]


In [18]:
X.shape

(26600, 768)

In [19]:
from sklearn.decomposition import PCA

decomposer_PCA = PCA(n_components = 10)
X = decomposer_PCA.fit_transform(X)
X.shape

(26600, 10)

In [20]:
from sklearn.model_selection import train_test_split as split

def train_test(X, y):

    X_trn, X_tst, y_trn, y_tst = split(X, y, test_size=0.4, random_state=1)
    X_val, X_tst, y_val, y_tst = split(X_tst, y_tst, test_size=0.5, random_state=1)

    return X_trn, X_tst, X_val, y_trn, y_tst, y_val

X_trn, X_tst, X_val, y_trn, y_tst, y_val = train_test(X,y)

print("Training data size:", len(X_trn))
print("Testing data size:", len(X_tst))
print("validation data size:", len(X_val))


Training data size: 15960
Testing data size: 5320
validation data size: 5320


In [21]:
X_trn

array([[-0.70828295,  0.73756504,  0.30305743, ..., -0.29736567,
        -0.19883478,  0.83663964],
       [-0.04206431,  0.3543415 , -0.46430445, ...,  0.40274304,
        -0.1435312 , -0.32086563],
       [ 0.11036003, -0.35619545,  0.03837967, ...,  0.00217843,
        -0.06637764, -0.5464809 ],
       ...,
       [ 0.09667075,  1.0173025 ,  0.27559757, ..., -0.05215734,
         0.06690311, -0.19657516],
       [-0.3080976 ,  0.5663209 , -0.2406677 , ...,  0.18957561,
        -0.3741467 , -0.4465406 ],
       [ 1.2101312 ,  0.25840473,  0.4460634 , ..., -0.11630952,
         0.06472516, -0.14368236]], dtype=float32)

In [22]:
X_tst

array([[-0.38108516, -0.75252056,  0.5016531 , ...,  0.7962749 ,
         0.21928477, -0.18669748],
       [-0.8558428 , -0.5249481 , -0.49484277, ..., -0.27708212,
         0.15845096, -0.1828785 ],
       [ 0.07074022,  0.23873854, -0.6619303 , ...,  0.0429666 ,
        -0.00374866,  0.64756167],
       ...,
       [-0.25434673, -1.0560522 ,  0.0279907 , ...,  0.00210556,
        -0.36381817,  0.20501614],
       [-0.03511369, -1.7168951 , -0.09120393, ...,  0.1145286 ,
         0.26173854,  0.14266396],
       [ 0.40141582,  0.84806204,  0.159608  , ..., -0.42286116,
         0.05236745,  0.03819394]], dtype=float32)

In [23]:
X_val

array([[-9.0201449e-01, -1.6306834e+00, -2.9049385e-01, ...,
         9.6222758e-04, -6.0080886e-02,  2.2360945e-01],
       [ 4.1716719e-01,  2.9479122e-01,  4.5710468e-01, ...,
        -2.8474355e-01,  6.6266060e-03, -2.1248138e-01],
       [-4.5389104e-01,  1.0386734e+00,  3.9453816e-01, ...,
        -2.0973516e-01, -8.5034370e-02,  1.7851782e-01],
       ...,
       [-1.1498814e+00,  6.7325687e-01,  3.4198105e-01, ...,
         1.5293956e-01,  2.9579413e-01, -4.7430992e-03],
       [ 1.1421204e-02,  8.8708067e-01,  1.9487095e-01, ...,
         6.4639759e-01,  1.9883704e-01,  7.7349544e-02],
       [-5.7636023e-01, -6.3289165e-02,  8.2834005e-02, ...,
         4.8833311e-02,  2.4713981e-01, -5.9767008e-02]], dtype=float32)

**Machine learning models (SVM, Random Forest, Logistic Regression)**

In [24]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import GridSearchCV as Grid
from sklearn.metrics import accuracy_score as Score
from sklearn.metrics import classification_report as Report

SVM

In [25]:
grid_parameter= {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

svm_mod = SVC()

grid_var = Grid(svm_mod, grid_parameter, cv=2, verbose=1)
grid_var.fit(X_trn, y_trn)

print(f"Best Parameters: {grid_var.best_params_}")
svm_mod = grid_var.best_estimator_

Y_pred = svm_mod.predict(X_val)
accuracy = Score(y_val, Y_pred)
Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Validation Score: {accuracy:.4f}")
print(Report(y_val, Y_pred))
print("Macro_F1_score", Macro_F1_score)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
Best Parameters: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}


, Validation Score: 0.7047
              precision    recall  f1-score   support

           0       0.72      0.78      0.75      3015
           1       0.68      0.60      0.64      2305

    accuracy                           0.70      5320
   macro avg       0.70      0.69      0.69      5320
weighted avg       0.70      0.70      0.70      5320

Macro_F1_score 0.6943047520890515


In [26]:
Y_pred = svm_mod.predict(X_tst)
accuracy = Score(y_tst, Y_pred)
Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Testing Score: {accuracy:.4f}")
print(Report(y_tst, Y_pred))
print("Macro_F1_score", Macro_F1_score)



, Testing Score: 0.7045
              precision    recall  f1-score   support

           0       0.71      0.79      0.75      2986
           1       0.69      0.60      0.64      2334

    accuracy                           0.70      5320
   macro avg       0.70      0.69      0.69      5320
weighted avg       0.70      0.70      0.70      5320

Macro_F1_score 0.6946510034297652


Random Forest

In [27]:
grid_parameter= {
    'n_estimators': [50, 10, 20],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

RF_mod = RF()

grid_var = Grid(RF_mod, grid_parameter, cv=2, verbose=1)
grid_var.fit(X_trn, y_trn)

print(f"Best Parameters: {grid_var.best_params_}")
RF_mod = grid_var.best_estimator_

Y_pred = RF_mod.predict(X_val)
accuracy = Score(y_val, Y_pred)
Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Validation Score: {accuracy:.4f}")
print(Report(y_val, Y_pred))
print("Macro_F1_score", Macro_F1_score)


Fitting 2 folds for each of 36 candidates, totalling 72 fits
Best Parameters: {'max_depth': None, 'min_samples_split': 10, 'n_estimators': 50}


, Validation Score: 0.6910
              precision    recall  f1-score   support

           0       0.71      0.77      0.74      3015
           1       0.66      0.58      0.62      2305

    accuracy                           0.69      5320
   macro avg       0.69      0.68      0.68      5320
weighted avg       0.69      0.69      0.69      5320

Macro_F1_score 0.680030953228314


In [28]:
Y_pred = RF_mod.predict(X_tst)
accuracy = Score(y_tst, Y_pred)
Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Testing Score: {accuracy:.4f}")
print(Report(y_tst, Y_pred))
print("Macro_F1_score", Macro_F1_score)



, Testing Score: 0.6921
              precision    recall  f1-score   support

           0       0.71      0.78      0.74      2986
           1       0.67      0.59      0.63      2334

    accuracy                           0.69      5320
   macro avg       0.69      0.68      0.68      5320
weighted avg       0.69      0.69      0.69      5320

Macro_F1_score 0.6819638027876014


Logistic Regression

In [29]:
grid_parameter= {
    'C': [0.01, 0.1, 1],
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'max_iter': [10, 20, 30]
}

LR_mod = LR()

grid_var = Grid(LR_mod, grid_parameter, cv=2, verbose=1)
grid_var.fit(X_trn, y_trn)

print(f"Best Parameters: {grid_var.best_params_}")
LR_mod = grid_var.best_estimator_

Y_pred = LR_mod.predict(X_val)
accuracy = Score(y_val, Y_pred)
Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Validation Score: {accuracy:.4f}")
print(Report(y_val, Y_pred))
print("Macro_F1_score", Macro_F1_score)


Fitting 2 folds for each of 27 candidates, totalling 54 fits
Best Parameters: {'C': 1, 'max_iter': 10, 'solver': 'lbfgs'}


, Validation Score: 0.6626
              precision    recall  f1-score   support

           0       0.68      0.75      0.72      3015
           1       0.63      0.55      0.58      2305

    accuracy                           0.66      5320
   macro avg       0.66      0.65      0.65      5320
weighted avg       0.66      0.66      0.66      5320

Macro_F1_score 0.6500558058638282


In [30]:
Y_pred = LR_mod.predict(X_tst)
accuracy = Score(y_tst, Y_pred)
Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

print(f"\n\n, Testing Score: {accuracy:.4f}")
print(Report(y_tst, Y_pred))
print("Macro_F1_score", Macro_F1_score)



, Testing Score: 0.6703
              precision    recall  f1-score   support

           0       0.69      0.75      0.72      2986
           1       0.64      0.57      0.60      2334

    accuracy                           0.67      5320
   macro avg       0.66      0.66      0.66      5320
weighted avg       0.67      0.67      0.67      5320

Macro_F1_score 0.6605869258413234


**Deep learning models (LSTM, BiLSTM)**

In [31]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
import numpy as np

LSTM

In [32]:
# hyper tuning the optimizer
for optimizer in ['adam', 'adamax', 'adagrad']:
  print("Result of hyper tuning the optimizer ", optimizer)

  LSTM_mod = Sequential()
  LSTM_mod.add(LSTM(200, input_shape=(X_trn.shape[1], 1)))
  LSTM_mod.add(Dense(1, activation='sigmoid'))
  LSTM_mod.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  LSTM_mod.fit(X_trn, y_trn, epochs=20, batch_size=128)

  Y_pred = LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the optimizer  adam
Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.5636 - loss: 0.6713
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.6237 - loss: 0.6459
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - accuracy: 0.6269 - loss: 0.6446
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.6294 - loss: 0.6432
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - accuracy: 0.6345 - loss: 0.6407
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.6160 - loss: 0.6455
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.6325 - loss: 0.6355
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.6324 - loss: 0.6334
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - accuracy: 0.6418 - loss: 0.6241
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.6606 - loss: 0.6187
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - accuracy: 0.6779 - loss: 0.60

In [33]:
# hyper tuning the count of epoches
for Epoches in [5, 10, 20]:
  print("Result of hyper tuning the count of epoches ", Epoches)

  LSTM_mod = Sequential()
  LSTM_mod.add(LSTM(200, input_shape=(X_trn.shape[1], 1)))
  LSTM_mod.add(Dense(1, activation='sigmoid'))
  LSTM_mod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  LSTM_mod.fit(X_trn, y_trn, epochs=Epoches, batch_size=128)

  Y_pred = LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the count of epoches  5
Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.5811 - loss: 0.6692
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.6245 - loss: 0.6452
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.6229 - loss: 0.6462
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.6271 - loss: 0.6416
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - accuracy: 0.6188 - loss: 0.6459
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


 Validation Score: 0.6323
              precision    recall  f1-score   support

           0       0.65      0.76      0.70      3015
           1       0.60      0.47      0.53      2305

    accuracy                           0.63      5320
   macro avg       0.62      0.61      0.61      5320
weighted avg       0.63      0.63      0.62      5320

Macro_F1_score 0.6128283896082132
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


 Testing Score: 0.6293
              precision    

BiLSTM

In [34]:
X_trn = np.expand_dims(X_trn, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_tst = np.expand_dims(X_tst, axis=-1)

In [35]:
# hyper tuning the optimizer
for optimizer in ['adam', 'adamax', 'adagrad']:
  print("Result of hyper tuning the optimizer ", optimizer)

  bi_LSTM_mod = Sequential()
  bi_LSTM_mod.add(Bidirectional(LSTM(200, input_shape=(X_trn.shape[1], 1))))
  bi_LSTM_mod.add(Dense(1, activation='sigmoid'))
  bi_LSTM_mod.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  bi_LSTM_mod.fit(X_trn, y_trn, epochs=20, batch_size=128)

  Y_pred = bi_LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = bi_LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the optimizer  adam
Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 81ms/step - accuracy: 0.5801 - loss: 0.6707
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - accuracy: 0.6201 - loss: 0.6491
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - accuracy: 0.6353 - loss: 0.6378
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 75ms/step - accuracy: 0.6700 - loss: 0.6069
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 82ms/step - accuracy: 0.6859 - loss: 0.5986
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - accuracy: 0.6813 - loss: 0.5967
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 68ms/step - accuracy: 0.6780 - loss: 0.5964
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - accuracy: 0.6867 - loss: 0.5936
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - accuracy: 0.6907 - loss: 0.5880
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.6895 - loss: 0.5904
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 72ms/step - accuracy: 0.7049 - los

In [36]:
# hyper tuning the count of epoches
for Epoches in [5, 10, 20]:
  print("Result of hyper tuning the count of epoches ", Epoches)

  bi_LSTM_mod = Sequential()
  bi_LSTM_mod.add(Bidirectional(LSTM(200, input_shape=(X_trn.shape[1], 1))))
  bi_LSTM_mod.add(Dense(1, activation='sigmoid'))
  bi_LSTM_mod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  bi_LSTM_mod.fit(X_trn, y_trn, epochs=Epoches, batch_size=128)

  Y_pred = bi_LSTM_mod.predict(X_val)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_val, Y_pred)
  Macro_F1_score = Report(y_val, Y_pred, output_dict=True)['macro avg']['f1-score']
  print(f"\n\n Validation Score: {accuracy:.4f}")
  print(Report(y_val, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

  Y_pred = bi_LSTM_mod.predict(X_tst)
  Y_pred = Y_pred.flatten()
  Y_pred = np.where(Y_pred > 0.5, 1, 0).astype('int')
  accuracy = Score(y_tst, Y_pred)
  Macro_F1_score = Report(y_tst, Y_pred, output_dict=True)['macro avg']['f1-score']

  print(f"\n\n Testing Score: {accuracy:.4f}")
  print(Report(y_tst, Y_pred))
  print("Macro_F1_score", Macro_F1_score)

Result of hyper tuning the count of epoches  5
Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - accuracy: 0.5868 - loss: 0.6705
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 70ms/step - accuracy: 0.6245 - loss: 0.6445
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 105ms/step - accuracy: 0.6214 - loss: 0.6457
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.6510 - loss: 0.6235
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 69ms/step - accuracy: 0.6756 - loss: 0.6025
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step


 Validation Score: 0.6855
              precision    recall  f1-score   support

           0       0.69      0.80      0.74      3015
           1       0.67      0.53      0.60      2305

    accuracy                           0.69      5320
   macro avg       0.68      0.67      0.67      5320
weighted avg       0.68      0.69      0.68      5320

Macro_F1_score 0.669225531084314
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step


 Testing Score: 0.6859
              precis